In [1]:
def drawPolyline(im, landmarks, start, end, isClosed=False) :
    points = []
    for i in range(start, end+1) :
        point = [landmarks.part(i).x, landmarks.part(i).y]
        points.append(point)
        
    points = np.array(points, dtype=np.int32)
    cv2.polylines(im, [points], isClosed, (255, 200, 0),
                 thickness=2, lineType=cv2.LINE_8)
    

# Use this function for 70-poinst facial lanmark detector model
def renderFace(im, landmarks) :
    assert(landmarks.num_parts == 68)
    drawPolyline(im, landmarks, 0, 16)              # Jaw line
    drawPolyline(im, landmarks, 17, 21)             # Left eyebrow
    drawPolyline(im, landmarks, 22, 26)             # Right eyebrow
    drawPolyline(im, landmarks, 27, 30)             # NOse bridge 
    drawPolyline(im, landmarks, 30, 35, True)       # Lower nose
    drawPolyline(im, landmarks, 36, 41, True)       # Left eye
    drawPolyline(im, landmarks, 42, 47, True)       # Right eye
    drawPolyline(im, landmarks, 48, 59, True)       # Outer LIp
    drawPolyline(im, landmarks, 60, 67, True)       # Inner lip

    
# Ues this funciton for any model other than 
# 70 points facial_landmark detctor model
def renderFace2(im, landmarks, color=(0, 255, 0), radius=3):
    for p in landmarks.parts() :
        cv2.circle(im, (p.x, p.y), radius, color, -1)

In [123]:
import dlib, cv2
import numpy as np
# from renderFace import renderFace

def writeLandmarksToFile(landmarks, landmarksFileName) :
    with open(landmarksFileName, 'w') as f:
        for p in landmarks.parts() :
            f.write("%s %s\n" %(int(p.x), int(p.y)))
            
    f.close()
    
def appendLandmarksToList(landmarks, arr) :
    for p in landmarks.parts() :
        arr.append((int(p.x), int(p.y)))
    

    
# Landmark model location
PREDICTOR_PATH = "./models/shape_predictor_68_face_landmarks.dat"

# Get the face detector
faceDetector = dlib.get_frontal_face_detector()

# The landmark detector is implemented in the shape_predictor class
landmarkDetector = dlib.shape_predictor(PREDICTOR_PATH)

    
    
# Read image
imageFilename1 = "./data/images/rapunzel.jpg"
imageFilename2 = "./data/images/seoyoung.jpg"
im1 = cv2.imread(imageFilename1)
# im2 = im1[:, int(im1.shape[1]/2) :]
# im1 = im1[:, :int(im1.shape[1]/2)]
im2 = cv2.imread(imageFilename2)
im1 = cv2.resize(im1, (600, 600))
im2 = cv2.resize(im2, (600, 600))
# im1 = cv2.flip(im1, 1)
# im2 = cv2.flip(im2, 1)

# Detect faces in the image
faceRects1 = faceDetector(im1, 0)
faceRects2 = faceDetector(im2, 0)
print("Number of faces 1 dectected: ", len(faceRects1))
print("Number of faces 2 dectected: ", len(faceRects2))

# List to store landmarks of all detected faces
landmarksAll1 = []
landmarksAll2 = []

# 랜드마크 찾기
# Loop over all detected face rectangles
for i in range(0, len(faceRects1)) :
    newRect1 = dlib.rectangle(int(faceRects1[i].left()),
                            int(faceRects1[i].top()),
                            int(faceRects1[i].right()),
                            int(faceRects1[i].bottom()))
    newRect2 = dlib.rectangle(int(faceRects2[i].left()),
                            int(faceRects2[i].top()),
                            int(faceRects2[i].right()),
                            int(faceRects2[i].bottom()))
    #
    # For every face rectangle, run landmarkDectector
    landmarks1 = landmarkDetector(im1, newRect1)
    landmarks2 = landmarkDetector(im2, newRect2)
    # Print number of landmarks
    if i==0:
        print("Number of landmarks 1 : ", len(landmarks1.parts()))
        print("Number of landmarks 2 : ", len(landmarks1.parts()))
        
    # Store landmarks for current face
    appendLandmarksToList(landmarks1, landmarksAll1)
    appendLandmarksToList(landmarks2, landmarksAll2)
    
#     Draw landmarks on face
#     renderFace(im1, landmarks1)
#     renderFace(im2, landmarks2)
    
#     landmarksFileName = imageFilename + ".txt"
#     print("Saving landmarks to", landmarksFileName)
    # Write landmarks to disk
#     writeLandmarksToFile(landmarks, landmarksFileName)

Number of faces 1 dectected:  1
Number of faces 2 dectected:  1
Number of landmarks 1 :  68
Number of landmarks 2 :  68


In [87]:
cropped_img.shape

(600, 600, 4)

array([255], dtype=uint8)

In [124]:
tmp = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
r, g, b = cv2.split(im2)
_,a = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
ddd = cv2.merge([r,g,b,a],4)
im2 = ddd
cv2.imshow("cropped", ddd)

cv2.waitKey()
cv2.destroyAllWindows()

In [89]:
tmp = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
r, g, b = cv2.split(im1)
_,a = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
ddd = cv2.merge([r,g,b,a],4)
im1 = ddd

In [92]:
landmarks = np.array(landmarksAll2)

outline = landmarks[[*range(17), *range(26,16,-1)]]
outline = np.array(outline)
cropped_img = np.zeros((im2.shape[0],im2.shape[1],4))
cv2.fillConvexPoly(cropped_img, cv2.convexHull(outline), (255,255,255,255))
cv2.imshow("cropped", cropped_img)
cv2.waitKey()
cv2.destroyAllWindows()

In [135]:
# out_face = np.zeros_like(im2)
out_face = np.copy(im2)
mask1 = cropped_img.astype(np.bool)
mask1 = np.invert(mask1)
# out_face[mask1] = 0

# im2 = out_face
cv2.imshow("cropped", out_face)
cv2.waitKey()
cv2.destroyAllWindows()

In [142]:
cv2.imshow("cropped", mask1.astype(np.int))
cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.1.1) C:/projects/opencv-python/opencv/modules/highgui/src/precomp.hpp:131: error: (-215:Assertion failed) src_depth != CV_16F && src_depth != CV_32S in function 'convertToShow'


In [ ]:
# 검은색 화면 

In [158]:
im2[3]

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       ...,
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=uint8)

#### 마우스로 포인트 추가 등록

In [97]:
# 귀 가로 끝 > 목 > 왼쪽 어깨 > 오른쪽 어깨 순으로
# 1. 마우스 이벤트 발생시 호출될 함수를 정의합니다. 
mouse = []
def mouse_callback1(event, x, y, flags, param):
    l = len(landmarksAll1)
    if event == cv2.EVENT_LBUTTONDOWN:
#         if (l==70):
#             x = im1.shape[1] - 1
#         if (l==71):
#             x = 0
        print("클릭 좌표 : %d, %d"%(x,y))
#         landmarksAll1.append((x,y))
        mouse.append((x,y))


# img = np.zeros((512, 512, 3), np.uint8)
cv2.namedWindow('image')  # 2. 마우스 이벤트를 감지할 윈도우를 생성합니다.  


# 3. 이름이 image인 윈도우에서 마우스 이벤트가 발생하면 mouse_callback 함수가 호출되게 됩니다. 
cv2.setMouseCallback('image', mouse_callback1)  


cv2.imshow('image',im1)
cv2.waitKey(0)

cv2.destroyAllWindows() 

landmarksAll1[17:27] = mouse[0:10]
landmarksAll1[36:48] = mouse[10:]

클릭 좌표 : 111, 202
클릭 좌표 : 125, 196
클릭 좌표 : 141, 196
클릭 좌표 : 159, 202
클릭 좌표 : 170, 208
클릭 좌표 : 236, 211
클릭 좌표 : 267, 210
클릭 좌표 : 294, 210
클릭 좌표 : 312, 214
클릭 좌표 : 333, 216
클릭 좌표 : 111, 262
클릭 좌표 : 123, 237
클릭 좌표 : 165, 246
클릭 좌표 : 174, 289
클릭 좌표 : 150, 295
클릭 좌표 : 117, 286
클릭 좌표 : 248, 294
클릭 좌표 : 269, 250
클릭 좌표 : 312, 252
클릭 좌표 : 351, 289
클릭 좌표 : 324, 315
클릭 좌표 : 278, 318


In [114]:
# 2번 사진
def mouse_callback2(event, x, y, flags, param):
    l = len(landmarksAll2)
    if event == cv2.EVENT_LBUTTONDOWN:
#         if (l==70):
#             x = im2.shape[1] - 1
#         if (l==71):
#             x = 0
        print("클릭 좌표 : %d, %d"%(x,y))
        landmarksAll2.append((x,y))


# img = np.zeros((512, 512, 3), np.uint8)
cv2.namedWindow('image')  # 2. 마우스 이벤트를 감지할 윈도우를 생성합니다.  


# 3. 이름이 image인 윈도우에서 마우스 이벤트가 발생하면 mouse_callback 함수가 호출되게 됩니다. 
cv2.setMouseCallback('image', mouse_callback2)  


cv2.imshow('image',im2)
cv2.waitKey(0)

cv2.destroyAllWindows() 

In [48]:
len(landmarksAll1), len(landmarksAll2)

(68, 68)

#### 이미지의 가장자리 8개 추가 등록

In [98]:
# 1 2 3 
# 8   4
# 7 6 5

for arr, im in [(landmarksAll1, im1), (landmarksAll2, im2)] :
    arr.append((0, 0))
    arr.append(((im.shape[1]-1)/2, 0))
    arr.append((im.shape[1]-1, 0))
    arr.append((im.shape[1]-1, (im.shape[0]-1)/2))
    arr.append((im.shape[1]-1, im.shape[0]-1))
    arr.append(((im.shape[1]-1)/2, im.shape[0]-1))
    arr.append((0, im.shape[0]-1))
    arr.append((0, (im.shape[0]-1)/2))

In [99]:
len(landmarksAll1), len(landmarksAll2)

(76, 76)

In [12]:
landmarksAll1[-8:]

[(0, 0),
 (199.5, 0),
 (399, 0),
 (399, 229.5),
 (399, 459),
 (199.5, 459),
 (0, 459),
 (0, 229.5)]

In [113]:
#!/usr/bin/env python

import numpy as np
import cv2
import sys

# Read points from text file
def readPoints(path) :
    # Create an array of points.
    points = [];
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))

    return points

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst


# Warps and alpha blends triangular regions from img1 and img2 to img
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))


    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []


    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 4), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0,1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)
    

    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2

    # Copy triangular region of the rectangular patch to the output image
    try :
        img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask
    except :
        print(( 1 - mask ).shape, "띠띠")
        print(img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]].shape, "빵빵")
        raise Exception

    return mask
    

In [114]:
# Check if a point is inside a rectangle
def rectContains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True


def calculateDelaunayTriangles(rect, points):
    # Create subdiv
    subdiv = cv2.Subdiv2D(rect)
   
    # Insert points into subdiv
    for p in points:
        subdiv.insert((p[0], p[1]))

   
    # List of triangles. Each triangle is a list of 3 points ( 6 numbers )
    triangleList = subdiv.getTriangleList()

    # Find the indices of triangles in the points array

    delaunayTri = []
    
    for t in triangleList:
        pt = []
        pt.append((t[0], t[1]))
        pt.append((t[2], t[3]))
        pt.append((t[4], t[5]))
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])        
        
        if rectContains(rect, pt1) and rectContains(rect, pt2) and rectContains(rect, pt3):
            ind = []
            for j in range(0, 3):
                for k in range(0, len(points)):                    
                    if(abs(pt[j][0] - points[k][0]) < 1.0 and abs(pt[j][1] - points[k][1]) < 1.0):
                        ind.append(k)                            
            if len(ind) == 3:                                                
                delaunayTri.append((ind[0], ind[1], ind[2]))
        

    
    return delaunayTri


In [115]:
# filename1 = './data/images/ben.jpg'
# filename2 = './data/images/morgan.jpg'
alpha = 0.5

# Read images
# img1 = cv2.imread(filename1);
# img2 = cv2.imread(filename2);
img1 = im1
img2 = im2

# Convert Mat to float data type
img1 = np.float32(img1)
img2 = np.float32(img2)

# Read array of corresponding points
#     points1 = readPoints(filename1 + '.txt')
#     points2 = readPoints(filename2 + '.txt')
points1 = landmarksAll1
points2 = landmarksAll2
points = [];

# Compute weighted average point coordinates
for i in range(0, len(points1)):
    x = ( 1 - alpha ) * points1[i][0] + alpha * points2[i][0]
    y = ( 1 - alpha ) * points1[i][1] + alpha * points2[i][1]
    points.append((x,y))


In [116]:
im2.shape
im1.shape

(600, 600, 4)

In [117]:
im2.shape

(600, 600, 4)

In [118]:

avgPoints = []
for a, b in zip(points1, points2) :
    avgPoints.append(((a[0]+b[0])/2 , (a[1]+b[1])/2))
avgPoints
print((0,0,im1.shape[1],im1.shape[0]))
dt = calculateDelaunayTriangles((0,0,im2.shape[1],im2.shape[0]),avgPoints)
dt

(0, 0, 600, 600)


[(17, 37, 0),
 (37, 17, 18),
 (2, 75, 1),
 (75, 2, 3),
 (75, 17, 0),
 (17, 75, 68),
 (29, 39, 28),
 (39, 29, 40),
 (0, 1, 75),
 (1, 0, 41),
 (31, 2, 40),
 (2, 31, 3),
 (74, 3, 4),
 (3, 74, 75),
 (2, 1, 40),
 (54, 47, 13),
 (47, 54, 35),
 (74, 4, 5),
 (49, 32, 50),
 (32, 49, 31),
 (73, 74, 7),
 (4, 3, 31),
 (67, 59, 61),
 (59, 67, 58),
 (74, 5, 6),
 (5, 4, 48),
 (67, 62, 66),
 (62, 67, 61),
 (74, 6, 7),
 (6, 5, 59),
 (56, 8, 57),
 (8, 56, 9),
 (7, 6, 58),
 (29, 35, 30),
 (35, 29, 42),
 (73, 7, 8),
 (8, 7, 57),
 (10, 73, 9),
 (73, 10, 11),
 (9, 73, 8),
 (11, 72, 73),
 (72, 11, 12),
 (10, 9, 55),
 (40, 29, 31),
 (72, 12, 13),
 (11, 10, 54),
 (21, 27, 38),
 (27, 21, 22),
 (12, 11, 54),
 (38, 40, 41),
 (40, 38, 39),
 (72, 13, 71),
 (13, 12, 54),
 (41, 0, 36),
 (70, 71, 16),
 (13, 14, 71),
 (14, 13, 46),
 (38, 20, 21),
 (20, 38, 37),
 (15, 71, 14),
 (71, 15, 16),
 (15, 14, 46),
 (37, 18, 19),
 (16, 15, 45),
 (69, 18, 68),
 (18, 69, 19),
 (0, 37, 36),
 (17, 68, 18),
 (35, 42, 47),
 (69, 20, 1

In [119]:
mmm

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 1., 1., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 0.],
        

In [120]:
# Allocate space for final output
imgMorph = np.zeros(img1.shape, dtype = img1.dtype)

# Read triangles from tri.txt
# with open("./data/images/tri.txt") as file :
    
for line in dt :
#     x,y,z = line.split()
    x = line[0] 
    y = line[1]
    z = line[2]
    
    x = int(x)
    y = int(y)
    z = int(z)

    t1 = [points1[x], points1[y], points1[z]]
    t2 = [points2[x], points2[y], points2[z]]
    t = [ points[x], points[y], points[z] ]

    # Morph one triangle at a time.
    morphTriangle(img1, img2, imgMorph, t1, t2, t, alpha)




In [108]:
imgMorph.shape

(600, 600, 4)

In [69]:
imgMorph[:,:,3] = np.full(imgMorph[:,:,3].shape, 255)

In [121]:
# Display Result
cv2.imshow("Morphed Face", np.uint8(imgMorph))
cv2.waitKey(0)
cv2.destroyAllWindows() 

# 오른 귀가 보이는 모습으로의 tri.txt가 되어있음
- 따라서 왼쪽 귀가 보이는 얽굴이면 좌우 반전 해줘야 될 것 같음

<code>cv2.flip(im1, 1)</code>
<code>cv2.flip(im2, 1)</code>
를 추가해야됨